In [ ]:
import scanpy as sc
import anndata as ad
import numpy as np
import seaborn as sns
from collections import Counter
import random

In [ ]:
adata = sc.read("real-data/data/1_raw/kang2018.h5ad")

In [ ]:
adata = adata[adata.obs['cell'] == 'CD4 T cells']
adata = adata[adata.obs['multiplets'] == 'singlet']
adata.obs['ind'] = adata.obs['ind'].astype(str)

In [ ]:
adata.obs

In [ ]:
adatas = []
for sample in adata.obs['ind'].unique():
    adata_samp = adata[adata.obs['ind'] == sample]
    stim = adata_samp.obs['stim'].tolist()
    random.shuffle(stim)
    adata_samp.obs['stim_resamp'] = stim
    adatas.append(adata_samp)

In [ ]:
adata = ad.concat(adatas)

In [ ]:
adata.obs['Sample'] = adata.obs['ind'] + "-" + adata.obs['stim_resamp']
adata.obs = adata.obs.rename(columns={'ind': 'Batch', 'stim_resamp': 'Condition'})
adata.obs['Condition'] = adata.obs['Condition'].replace(to_replace=['ctrl', 'stim'], value = ['Condition1', 'Condition2'])

In [ ]:
adata.obs

In [ ]:
adata.obs['n_counts'] = adata.X.sum(axis = 1)
adata.obs['n_genes'] = (adata.X > 0).sum(axis=1)
mt_gene_mask = np.flatnonzero([gene.startswith('MT-') for gene in adata.var_names])
adata.obs['mt_frac'] = np.sum(adata[:, mt_gene_mask].X, axis=1).A1 / adata.obs['n_counts']

In [ ]:
adata.layers['counts'] = adata

In [ ]:
adata.write("real-data/data/1_raw/raw.h5ad")